<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gated-CNN" data-toc-modified-id="Gated-CNN-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gated CNN</a></span><ul class="toc-item"><li><span><a href="#GCNN-2D" data-toc-modified-id="GCNN-2D-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>GCNN 2D</a></span></li></ul></li></ul></div>

In [1]:
# IMPORTS
from src.model import *

SyntaxError: invalid syntax (hidrometer_consumption.py, line 23)

In [ ]:
dataload = Sass('/home/penalvad/stattus4/dnn_test_data/', num_samples=7, number_of_batches=1, split_tax=.9, freq_size=600, time_size=80, same_data_validation=True)
dataload_2 = Sass('/home/penalvad/stattus4/test/', num_samples=5, number_of_batches=1, split_tax=.9, freq_size=480, time_size=600, same_data_validation=True)

In [ ]:
dtrain = dataload.training()
dtrain_2 = dataload_2.training()

In [ ]:
data_train = np.ndarray( shape=(12,600,80,1) )
data_test = np.ndarray( shape=(8,600,50) )
for i in range(6):

    data_train[2*i,:,:,0] = dtrain[i][0][2]
    data_train[2*i+1,:,:,0] = dtrain[i][1][2]

# Architecture Basics
##  Gated CNN
Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.\n",
It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.

### GCNN 2D
This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:

Padding Valid
\\begin{align}
\dimemsion output size = \\ceil{ \\frac{dim size - (kernel dim size - 1)*dilation rate}{stride} }
\\end{align}

Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data).

In [ ]:
#Cascading deepness networks mem size

def channeling_rule(channelin, filterlist):
  channellist = [channelin]
  for filter in filterlist:
    channellist.append(channellist[-1] + filter + 3)

  return channellist

In [ ]:
# Label Vector Feed
# dtrain[0][0][1]
ltrain = []
ltest = []

for i in range(6):
    ltrain.append([0,1])
    ltrain.append([1,0])

for i in range(4):
    ltest.append([0,1])
    ltest.append([1,0])

In [ ]:
# Define Filter-Bank of the CNN 
filterlist = []
filterlista = [3,3,3,3]
filterlist.append(filterlista)
a = calculatefilter(600, filterlista)

# Max pooling

filterlistb = [5,5,5,5]
filterlist.append(filterlistb)
b = calculatefilter(a/2.0, filterlistb)

# Max pooling

filterlistc = [7,7,7,7]
filterlist.append(filterlistc)
c = calculatefilter(b/2., filterlistc)

# Max pooling 

filterlistd = [5,5,5,5]
filterlist.append(filterlistd)
d = calculatefilter(c/2.0, filterlistd)


# Max pooling 

filterliste = [5,4,4]
filterlist.append(filterliste)
e = calculatefilter(d/2.0, filterliste)
print(e)

In [ ]:
#Model Building

# Input Frames

#Frequency
freq=600*np.ones(20)

#Time 
time = np.arange(0,80,4) + 4


buildgraph = Builder(dtype=tf.float32, datasize=(freq,time), num_input = 20, channels=1)
buildgraph.get_directives('gcnn2d')
buildgraph.get_directives('softmax')
#buildgraph.set_archname('frame')
buildgraph.get_directives('reducemean')
buildgraph.get_directives('losscrossentropy')
pooling = [2,2,2,2]

for i in range(20):

    buildgraph.config_block_name(deepness=len(filterlist[0]),numblock=i)
    for j in np.arange(len(filterlist)):

        cout = 64 + j*20
        if j != 0:
            buildgraph.config_block_name(deepness=len(filterlist[j]),numblock=i)

        for z in np.arange(len(filterlist[j])):
            
            fw = (np.floor((np.arange(0,80,4) + 4)*0.03)+1)

            if z == 0 and j == 0:
                buildgraph.build_graph_module('cnn2d', channels_out = 64, filter_size=(filterlist[j][0], np.int(fw[19-i])), isinput=True, lastnamescope=True, verbose=False)

            # z:= layer onde esta o gate dentro de um block de cnn, entre maxpoolings.
            # j:= número do block de cnn, onde esta o layer z de gate
            if z ==  (len(filterlist[j])-1) and (j == 4):
                buildgraph.build_graph_module('gcnn2d', channels_out = cout, filter_size=(filterlist[j][z], np.int(fw[19-i])), isinput=False, lastnamescope=True, verbose=False)
            else:
                buildgraph.build_graph_module('cnn2d', channels_out = cout, filter_size=(filterlist[j][z], np.int(fw[19-i])), isinput=False, lastnamescope=True, verbose=False)
 
        if j < len(filterlist) - 1:
            buildgraph.config_block_name(deepness=-1,numblock=i,pool=j)                
            buildgraph.build_graph_module('maxpooling2d', poolsize = (pooling[j],1), isinput=False, lastnamescope=True, verbose=False)
  
    buildgraph.config_block_name(deepness=0,numblock=i)
    buildgraph.build_graph_module('softmax', num_labels=2, isinput=False, lastnamescope=True)

buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('reducemean', isinput=False, lastnamescope=True)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('losscrossentropy', num_labels = 2, isinput=False, lastnamescope=True, show_cgraph=False)

In [ ]:
print_display_cgraph(buildgraph.graph)

In [ ]:
time

In [ ]:
feed_dict = {}
for i in range(20):    
    feed_dict[buildgraph.signal_in[i]] = data_train[:12, :, :time[19-i], :]
    feed_dict[buildgraph.label_tensor] = ltrain[:12]
    feed_dict[buildgraph.learning_rate] = 0.00001
output = buildgraph.run_cgraph(feed_dict, op_to_run =buildgraph.namescopes['reducemean'][-1].name, number_of_runs = 250, mode = 'minimize', ckpt_dir_name='../ckpt/model', new_session=True, output_log = True, stop=0.4, adaptative_lr=True, verbose=True)


In [ ]:
output[180]

In [ ]:
# Example of Recovering Model from .meta and .data. It uses .index and checkpoint to map variables and last checkpoint to the graph \n",
graph = tf.Graph()
    
with graph.as_default():
    
    saver = tf.train.import_meta_graph(\"../ckpt/model.meta\", import_scope='')
    feed_dict = {}
    with tf.Session(graph=graph) as sess:


        saver.restore(sess, tf.train.latest_checkpoint('../ckpt/') )    \n",
        feed_dict[sess.graph.get_tensor_by_name('signal_in:0')] = data_train[:14,:,:8,:]\n",
        for i in np.arange(1,10):  \n",
    "      feed_dict[sess.graph.get_tensor_by_name('signal_in_'+str(i)+':0')] = data_train[:14,:,8*i:8*(i+1),:]\n",
    "    feed_dict[sess.graph.get_tensor_by_name('losscrossentropy/labels:0')] = ltrain[:14]\n",
    "    feed_dict[sess.graph.get_tensor_by_name('learning_rate:0')] = 0.0001      \n",
    "  #for op in graph.get_operations():\n",
    "    #print(2*'\\n')\n",
    "    #print(op.name)  \n",
    "    minimize_op = sess.graph.get_operation_by_name('losscrossentropy/Adam')\n",
    "    loss = sess.graph.get_tensor_by_name('losscrossentropy/categorical_crossentropy/weighted_loss/value:0')\n",
    "    acc = sess.graph.get_tensor_by_name('reducemean/Mean:0')\n",
    "    for i in range(400):\n",
    "      lossval,_ = sess.run([loss,minimize_op], feed_dict=feed_dict)\n",
    "      accval = sess.run([acc], feed_dict=feed_dict)\n",
          if lossval < 0.3:\n",
            saver.save(sess, '../ckpt/model2')\n",
            print('\\n')\n",
            print('saving')\n",
            print('\\n')\n",
            \n",
          print('loss ',lossval)     \n",
          print('acc ', accval)\n",
          print(2*'\\n')\n",
    
